In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [4]:
train = pd.read_csv("./Train.csv")##.values
test = pd.read_csv("./Test.csv")##.values

In [5]:
XT = train['review'].values
Xt = test['review'].values
YT = train['label'].values

In [6]:
for i in range(YT.shape[0]) :
    if YT[i]=='pos':
        YT[i] = 1
    else :
        YT[i] = 0
    

In [7]:
YT

array([1, 1, 1, ..., 0, 1, 1], dtype=object)

In [8]:
print(XT.shape,YT.shape)

(40000,) (40000,)


In [9]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(XT)
sequences = tokenizer.texts_to_sequences(XT)
XT = pad_sequences(sequences, maxlen=50)

In [10]:
sequences = tokenizer.texts_to_sequences(Xt)
Xt = pad_sequences(sequences, maxlen=50)

In [11]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           2000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 2,080,501
Trainable params: 2,080,501
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(XT,YT, validation_split=0.4,batch_size=128,epochs=7,shuffle = True)

Train on 24000 samples, validate on 16000 samples
Epoch 1/7
24000/24000 [==============================] - 34s 1ms/step - loss: 0.2878 - acc: 0.8824 - val_loss: 0.4310 - val_acc: 0.8189
Epoch 2/7
24000/24000 [==============================] - 34s 1ms/step - loss: 0.2321 - acc: 0.9101 - val_loss: 0.4592 - val_acc: 0.8099
Epoch 3/7
24000/24000 [==============================] - 29s 1ms/step - loss: 0.1839 - acc: 0.9300 - val_loss: 0.5532 - val_acc: 0.8027
Epoch 4/7
24000/24000 [==============================] - 30s 1ms/step - loss: 0.1488 - acc: 0.9456 - val_loss: 0.6018 - val_acc: 0.8028
Epoch 5/7
24000/24000 [==============================] - 29s 1ms/step - loss: 0.1169 - acc: 0.9572 - val_loss: 0.6145 - val_acc: 0.7999
Epoch 6/7
24000/24000 [==============================] - 29s 1ms/step - loss: 0.0899 - acc: 0.9680 - val_loss: 0.7045 - val_acc: 0.7919
Epoch 7/7
24000/24000 [==============================] - 29s 1ms/step - loss: 0.0689 - acc: 0.9771 - val_loss: 0.8853 - val_acc: 0.791

In [27]:
Yt = model.predict(Xt)

In [34]:
Yt

array([[3.696937e-02],
       [3.245487e-01],
       [5.645824e-04],
       ...,
       [9.994418e-01],
       [9.891879e-01],
       [7.778685e-02]], dtype=float32)

In [46]:
Y = []
for i in range(Yt.shape[0]) :
    if Yt[i]>=0.5:
        Y.append('pos')
    else :
        
        Y.append('neg')

In [52]:
Yt.shape

(10000, 1)

In [50]:
import csv
with open('Sample_Submission.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    m = Yt.shape[0]
    writer.writerow(["Id","label"])
    for i in range(m):
        
        l = [i+1,Y[i]]
        writer.writerow(l)
    csvFile.close()